In [ ]:
using System.Collections.Generic;
using UnityEngine;
using System;
namespace SkillSystem
{
public class Skill 
{
    public string SkillName ;
   public bool isActivated = false;
   public bool isReady {get{return _isReady;}set{_isReady = value;/*Debug.Log("set "+SkillName+" "+_isReady);*/}}
   private bool _isReady = true;
   public bool isAutoReset = true;
   public float Progress = 0;
   /// <summary>
    /// 只有当进度为1时才会为true,如果被中途打断则会为false
    /// </summary>
    public bool IsEnd{get{return  Progress>=1 ;}}
   //
   public GameEntity entity;
   public SkillManager manager;
   public List<ScheduleLine> scheduleLines = new List<ScheduleLine>();
   public List<ReadyLine> readyLines = new List<ReadyLine>();
    public List<SkillEffectBase> effects = new List<SkillEffectBase>();
    //
    public Skill(string skillname,GameEntity user,bool isAutoReset=false)
    {
        SkillName = skillname;
        entity = user;
        this.isAutoReset = isAutoReset;
    }
    public Skill(string skillname,GameEntity user,SkillEffectBase[] effects ,bool isAutoReset=false)
    {
        SkillName = skillname;
        entity = user;
        this.isAutoReset = isAutoReset;
        this.effects.AddRange(effects);
    }

    // 激活技能
    public void Activate()
    {   
        if (isActivated)
        {Debug.Log("已经actived");return;}
        if (isReady)
        {   isActivated = true;
            foreach (SkillEffectBase effect in effects)
            {
                effect.Apply(this);
            }
            //
            isReady = false;
        }
    }
    // 
    public void Update()
    {
        if (isActivated)
        {
            foreach (ReadyLine line in readyLines)
            {
                line.Update();
            }
            if(Progress>=1)
            {   
                End();
                return;
            }
            //注意这里的顺序
            foreach (SkillEffectBase effect in effects)
            {
                SetProgress(effect.Update());
            }
            
            
            foreach (ScheduleLine line in scheduleLines)
            {
                line.Update();
            }
            
        }
    }
    //
    public void End(){
        
        isActivated = false;
        Progress = 0;
        foreach (SkillEffectBase effect in effects)
        {
            effect.Stop();
        }
        if (isAutoReset)
        {
            isReady = true;
        }
        
    }
    //
    public void SetProgress(float progress)
    {
        if(progress>Progress)
        {
            Progress = progress;
            if (Progress >= 1)
            {
                Progress = 1;
            }
        }
    }
    //
    public void AddEffect(SkillEffectBase effect)
    {
        effects.Add(effect);
    }
    //
    public ScheduleLine ConnectTo(Skill to, Func<bool> trigger, bool isResetTargetReady=true, bool isEndFrom=false)
    {
        ScheduleLine line = new ScheduleLine(this, to, trigger, isResetTargetReady, isEndFrom);
        scheduleLines.Add(line);
        return line;
    }
    public ScheduleLine ConnectTo(Skill to, Trigger trigger, bool isResetTargetReady=true, bool isEndFrom=false)
    {
        ScheduleLine line = new ScheduleLine(this, to, trigger, isResetTargetReady, isEndFrom);
        scheduleLines.Add(line);
        return line;
    }

    public  ScheduleLine ConnectFromAlways(  SkillManager manager , Func<bool> trigger, bool isResetTargetReady = false)
        {
            ScheduleLine line = new ScheduleLine(null, this, trigger, isResetTargetReady, false);
            manager.AlwaysSceduleLines.Add(line);
            return line;
        }
    public  ScheduleLine ConnectFromAlways(  SkillManager manager , Trigger trigger, bool isResetTargetReady = false)
        {
            ScheduleLine line = new ScheduleLine(null, this, trigger, isResetTargetReady, false);
            manager.AlwaysSceduleLines.Add(line);
            return line;
        }
    //
    public ReadyLine ReadyTargetWhen(Skill target, Func<bool> trigger)
    {
        ReadyLine line = new ReadyLine(target, trigger);
        readyLines.Add(line);
        return line;
    }
    public ReadyLine ReadyTargetWhen(Skill target, Trigger trigger)
    {
        ReadyLine line = new ReadyLine(target, trigger);
        readyLines.Add(line);
        return line;
    }
    public ReadyLine ReadyTargetWhenEnd(Skill target)
    {
        ReadyLine line = new ReadyLine(target,this);
        readyLines.Add(line);
        return line;
    }

}

}
//Effect.cs
namespace SkillSystem
{
    
public abstract class SkillEffectBase 
{   
    protected Skill user;
    public virtual void Apply(Skill user){
        this.user = user;
        
    }
    public abstract float Update();

    public abstract void Stop();
}

}
//ScheduleLine.cs
namespace SkillSystem
{
public class ScheduleLine
{
    public Skill from;
    public Skill to;
    public Func<bool> AgentTrigger;//TODO:专门的trigger对象
    public Trigger trigger;
    public bool IsResetTargetReady = false;
    public bool IsEndFrom = false;
    //set
    public ScheduleLine SetIsResetTargetReady(bool isResetTargetReady)
    {
        IsResetTargetReady = isResetTargetReady;
        return this;
    }
    public ScheduleLine SetIsEndFrom(bool isEndFrom)
    {
        IsEndFrom = isEndFrom;
        return this;
    }
    //

        public ScheduleLine(Skill from, Skill to, Func<bool> trigger, bool isResetTargetReady, bool isEndFrom)
        {
            this.from = from;
            this.to = to;
            this.AgentTrigger = trigger;
            IsResetTargetReady = isResetTargetReady;
            IsEndFrom = isEndFrom;
        }

        public ScheduleLine(Skill from, Skill to, Trigger trigger, bool isResetTargetReady, bool isEndFrom)
        {
            this.from = from;
            this.to = to;
            this.trigger = trigger;
            AgentTrigger = GetFromTrigger;
            IsResetTargetReady = isResetTargetReady;
            IsEndFrom = isEndFrom;
        }
            private bool GetFromTrigger()=>trigger.IsTriggered();

        public void Update()
    {
        if (AgentTrigger.Invoke())
        {
            
            
            if (IsResetTargetReady)
            {
                
                to.isReady = true;
            }
            if (IsEndFrom&&null!=from)
            {
                from.End();
            }
        
            if(to.isReady)
            {
                
                to.Activate();
            }
        
        }
    }
}
/// <summary>
/// 用于设置技能的准备状态为true
/// 默认是在技能结束时设置为true
/// </summary>
public class ReadyLine
{
    public Skill to;
    public Func<bool> AgentTrigger;//TODO:专门的trigger对象
    public Trigger trigger;

    public ReadyLine(Skill to,Skill from){
        this.to = to;
        trigger = new SkillEndTrigger(from);
        AgentTrigger = GetFromTrigger;
    }
    public ReadyLine(Skill to, Func<bool> trigger)
    {
        this.to = to;
        this.AgentTrigger = trigger;
    }

    public ReadyLine(Skill to, Trigger trigger)
    {
        this.to = to;
        this.trigger = trigger;
        AgentTrigger = GetFromTrigger;
    }
    private bool GetFromTrigger()=>trigger.IsTriggered();

    public void Update()
    {
        if (AgentTrigger.Invoke()&&null!=to)
        {

            to.isReady = true;
        }
    }
}

}
//Parameter.cs

public class Parameter<T> where T : struct
{
    public T value{get { return AgentGetter == null ? defaultValue : AgentGetter.Invoke(); }}
    public T defaultValue;
    private Func<T> AgentGetter;
    private Getter<T> getter;
    public Parameter(T defaultValue, Func<T> agentGetter)
    {
        this.defaultValue = defaultValue;
        this.AgentGetter = agentGetter;
    }
    public Parameter(T defaultValue, Getter<T> getter)
    {
        this.defaultValue = defaultValue;
        this.getter = getter;
        this.AgentGetter = getFromGetter;
    }
        private T getFromGetter() => getter.Get();
    public Parameter(T defaultValue)
    {
        this.defaultValue = defaultValue;
    }
    public void SetAgentGetter(Func<T> agentGetter)
    {
        this.AgentGetter = agentGetter;
    }
    public void SetGetter(Getter<T> getter)
    {
        this.getter = getter;
        this.AgentGetter = getFromGetter;
    }
    public void Reset()
    {
        AgentGetter = null;
        getter = null;
    }

    public static implicit operator T(Parameter<T> parameter)
    {
        return parameter.value;
    }

    public static implicit operator Parameter<T>(T value)
    {
        return new Parameter<T>(value);
    }
    public static implicit operator Parameter<T>(Getter<T> getter)
    {
        return new Parameter<T>(default(T),getter);
    }
}

public class RefParameter<T> where T : class
{
    public T value{get { return AgentGetter == null ? defaultValue : AgentGetter.Invoke(); }}
    public T defaultValue;
    private Func<T> AgentGetter;
    private refGetter<T> getter;
    public RefParameter(T defaultValue, Func<T> agentGetter)
    {
        this.defaultValue = defaultValue;
        this.AgentGetter = agentGetter;
    }
    public RefParameter(T defaultValue, refGetter<T> getter)
    {
        this.defaultValue = defaultValue;
        this.getter = getter;
        this.AgentGetter = getFromGetter;
    }
        private T getFromGetter() => getter.Get();
    public RefParameter(T defaultValue)
    {
        this.defaultValue = defaultValue;
    }
    public void SetAgentGetter(Func<T> agentGetter)
    {
        this.AgentGetter = agentGetter;
    }
    public void SetGetter(refGetter<T> getter)
    {
        this.getter = getter;
        this.AgentGetter = getFromGetter;
    }
    public void Reset()
    {
        AgentGetter = null;
        getter = null;
    }

    public static implicit operator T(RefParameter<T> parameter)
    {
        return parameter.value;
    }

    public static implicit operator RefParameter<T>(T value)
    {
        return new RefParameter<T>(value);
    }
    public static implicit operator RefParameter<T>(refGetter<T> getter)
    {
        return new RefParameter<T>(default(T),getter);
    }
}
//Getter.cs
public abstract class Getter<T> :IGetter<T> where T : struct
{
    public abstract T Get();
}

public interface IGetter<T>
{
    T Get();
}
public abstract class refGetter<T> : IGetter<T>
{    
    public abstract T Get();
}
//Trigger.cs
namespace SkillSystem
{
    
    public interface ITrigger
{
    bool IsTriggered();
}
[Serializable]
public abstract class Trigger : Getter<bool>, ITrigger
{
    public bool IsTriggered(){
        return Get();
    }

    public static Trigger operator &(Trigger trigger1, Trigger trigger2)
        {
            return new AndTrigger(trigger1, trigger2);
        }
        public static Trigger operator |(Trigger trigger1, Trigger trigger2)
        {
            return new OrTrigger(trigger1, trigger2);
        }
        public static Trigger operator !(Trigger trigger)
        {
            return new NotTrigger(trigger);
        }
}



}



我想的大概是这样的:
###  1.类似节点编辑的形式(node和line)
###  2.我的技能系统的大概逻辑是这样的:
* _关键的类有:_
    * (1)Skill类:本身不含任何逻辑,包含SkillEffect类及SkillEffect子类的对象;包含由这个技能引出的调度线ScheduleLine和预备线ReadyLine;(skill的进度由每个effect的进度的最大)
    * (2)SillEffect类:功能由各个子类具体实现,子类中包含有不等的Parameter对象;
    * (3)ScheduleLine类:由Skill对象维护,内部有外部注入的Trigger对象,当触发时,调用Skill的Apply方法,(可以设置同时停止该Skill)
    * (4)ReadyLine类:由Skill对象维护,内部有外部注入的Trigger对象,当触发时,调用设置Target的IsReady属性为true(当IsReady为false时,即使有ScheduleLine触发,也不会Apply)
    * (5)Trigger类:具体由各个子类实现,具体类的创建需要参数
    * (6)Parameter<T> 和 RefParameter<T>类:用于存储参数,RefParameter<T>用于存储引用参数:可以设置DefaultValue,也可以注入Getter对象来进行复杂可变的参数获取
    * (7)Getter<T>类:用于获取参数,具体由各个子类实现,具体类的创建需要参数
* _创建逻辑:_
    创建一个Skill对象,然后创建各个SkillEffect对象,其中可以使用Parameter<T>和RefParameter<T>来填充参数,这里的Parameter也需要注入Getter对象来进行复杂可变的参数获取,然后将SkillEffect对象添加到Skill对象中,最后将Skill对象添加到SkillManager中,然后创建Trigger对象,并将其注入到ScheduleLine或ReadyLine中,最后连接ScheduleLines或ReadyLines连接各个Skill对象
    这样就完成了一个技能的创建.  
    __注意,在"Getter""Trigger""SillEffect"中除了会用到一些常量参数,还会用到一些对这个过程中创建的对象的引用,这些引用都是通过构造函数注入的,所以在创建这些对象时需要存下来,以便在创建其他对象时使用__
* _使用例子:_
    ```csharp

     var t = playerEntity.GetComponent<SkillSystem.SkillManager>();
        //创建技能
        var t_debug_skill1 = new SkillSystem.Skill("debug_skill1",playerEntity,isAutoReset:false);
        var t_shootSkillEffect1 =new SkillSystem.SkillEfect.shootSkillEffect(
            new Parameter<float>(0.5f),
            Instantiate(StartProjectileSO),
            Vector2.zero,
            10f,
            new Parameter<Vector2>(//这里是一个参数类,可以将"沉底"的参数传递出来,具体的方式是通过Getter类或者委托来获取参数的值
                Vector2.zero,
                new EntityVector2Getter(playerEntity, "DirectionToMouse")),
            1,
            30f,
            0.1f );
        t_debug_skill1.AddEffect(t_shootSkillEffect1);

        var t_debug_skill2 = new SkillSystem.Skill("debug_skill2",playerEntity,isAutoReset:false);
            var t_param_closeEntity=new RefParameter<GameEntity>(null);
        
        t_debug_skill2.AddEffect(new SkillSystem.SkillEfect.RefParameterDefaultChangeEffect<GameEntity>(
            t_param_closeEntity,
            (sk)=>{ return sk.entity.GetClosestEntity(30f);},
            (_)=>{return null;}
            ));
            SkillSystem.SkillEfect.shootSkillEffect t_shootSkillEffect2;
            t_shootSkillEffect2=new SkillSystem.SkillEfect.shootSkillEffect(
            new Parameter<float>(0.5f),
            Instantiate(StartProjectileSO),
            Vector2.zero,
            10f,
            new Parameter<Vector2>(//这里是一个参数类,可以将"沉底"的参数传递出来,具体的方式是通过Getter类或者委托来获取参数的值
                Vector2.zero,
                new EntityVector2Getter(playerEntity, "DirectionToMouse")),
            3,
            30f,
            0.1f,
            new ProjectileLogicSub[]{
                new TrackingLogicSub(t_param_closeEntity,30)
                });
        t_debug_skill2.AddEffect(t_shootSkillEffect2);
        
        t_debug_skill2.AddEffect(new SkillSystem.SkillEfect.ProjMovControlSkillEffect(
            new RefParameter<Projection[]>(null,
                t_shootSkillEffect1.CreateProjectionsGetter()),
                (proj)=>{Vector2 mousPos = Camera.main.ScreenToWorldPoint(Input.mousePosition);
                         return ((mousPos-(Vector2)proj.transform.position).normalized) *proj.DesireVelocity.magnitude;
                         }
                
        ));

        var t_debug_skill3 = new SkillSystem.Skill("debug_skill3",playerEntity,isAutoReset:false);
        t_debug_skill3.AddEffect(new SkillSystem.SkillEfect.shootSkillEffect(
            new Parameter<float>(1f),
            Instantiate(StartProjectileSO),
            Vector2.zero,
            10f,
            new Parameter<Vector2>(//这里是一个参数类,可以将"沉底"的参数传递出来,具体的方式是通过Getter类或者委托来获取参数的值
                Vector2.zero,
                new EntityVector2Getter(playerEntity, "DirectionToMouse")),
            3,
            30f,
            0));
        //添加技能
        t.Add(t_debug_skill1);
        t.Add(t_debug_skill2);
        t.Add(t_debug_skill3);
        //连接技能的调度
        t.ConnectAlways(t_debug_skill1,new SkillSystem.InputTrigger("Fire1")  );
        t_debug_skill1.ConnectTo(t_debug_skill2,new SkillSystem.InputTrigger("Fire1") &new SkillSystem.SkillProgressTrigger(t_debug_skill1,0.6f,1f),isEndFrom:true);
        t_debug_skill2.ConnectTo(t_debug_skill3,new SkillSystem.InputTrigger("Fire1") &new SkillSystem.SkillProgressTrigger(t_debug_skill2,0.6f,1),isEndFrom:true);//注意如果是false那么上一个技能的进度就会走完,从而触发ReadyTargetWhenEnd
        //连接技能的预备
        t_debug_skill1.ReadyTargetWhenEnd(t_debug_skill1);//自己连自己;如果没有在最后之前跳转，那么就会自动重置,反之则不会
        //t_debug_skill2.ReadyTargetWhenEnd(t_debug_skill2);//同上
        //t_debug_skill3.ReadyTargetWhenEnd(t_debug_skill3);//同上
        //之所以要用预备线,是因为如果直接用Skill的autoReset,即使在最后之前跳转,也会自动重置(具体是因为autoReset在End()函数中执行)
        t_debug_skill2.ReadyTargetWhenEnd(t_debug_skill1);//当第二个技能结束时，第一个技能就会准备好
        t_debug_skill3.ReadyTargetWhenEnd(t_debug_skill1);//同上

    ```

In [15]:
flowchart LR
a[[shootSkillEffect]]
p1(duration)-->a
p2(ProjectileSO)-->a
p3(PosOffset)-->a
p4(speed)-->a
p5(dir)-->a
p6(...)-->a
p7(ProjectileLogicSub)-->a

g1(EntityVector2Getter)-->p5
a<-->s1([Skill2])

您希望在节点编辑器中创建哪些类型的节点？例如，是否需要创建Skill、SkillEffect、ScheduleLine、ReadyLine、Trigger、Parameter、Getter、RefParameter等节点？

您希望在节点之间创建什么类型的连接？例如，是要创建Skill到SkillEffect的连接，还是其他节点之间的连接？

您希望在节点编辑器中提供哪些编辑功能？例如，您需要提供创建、删除、移动、缩放、连线、复制等功能。

您希望如何将创建的节点和连接保存和加载？例如，您希望使用XML、JSON或二进制格式保存和加载节点和连接。

您希望在编辑器中提供什么类型的视图和预览？例如，您需要在编辑器中显示节点和连接的属性和值，并提供一个预览视图来查看创建的技能的效果。

A:  
1. 我希望在节点编辑器中创建的节点有:Skill节点、SkillEffect节点、Trigger节点、Parameter节点、Getter节点、RefParameter节点
2. 我希望在节点之间创建的连接有:
    * Skill之间的两种连接,ScheduleLine和ReadyLine(其中ScheduleLine是用来连接技能的调度,ReadyLine是用来连接技能的预备;并且两个连接线可以连接Trigger);
    * Skill与SkillEffect节点的连接,表示拥有;
    * 具体SkillEffect节点与对应类型Parameter<T>和RefParameter<T>的连接;
    * 两种Parameter与对应Getter的连接,表示设置为Parameter的getter;
    * getter与Trigger 都可以与 其他所有这里创建的对象 连接,表示创建的具体getter与Trigger对象参数
3. 编辑功能现在只要求:创建、移动、连线
4. 暂时不需要(毕竟只作为技术展示)
5. 不需要预览

In [ ]:
// public static floatInPort FloatInputPort(float defaultValue)
//     {
//         GameObject port = GameObject.Instantiate(PortPrefab);
//         floatInPort portComponent = port.GetComponent<floatInPort>();
//         portComponent.defaultValue = defaultValue;
//         return portComponent;
//     }
//     public static floatOutPort FloatOutputPort(Func<Parameter<float>> parameterGetter)
//     {
//         GameObject port = GameObject.Instantiate(PortPrefab);
//         floatOutPort portComponent = port.GetComponent<floatOutPort>();
//         portComponent.ParameterGetter = parameterGetter;
//         return portComponent;
//     }
//     public static Vector2InPort Vector2InputPort(Vector2 defaultValue)
//     {
//         GameObject port = GameObject.Instantiate(PortPrefab);
//         Vector2InPort portComponent = port.GetComponent<Vector2InPort>();
//         portComponent.defaultValue = defaultValue;
//         return portComponent;
//     }
//     public static Vector2OutPort Vector2OutputPort(Func<Parameter<Vector2>> parameterGetter)
//     {
//         GameObject port = GameObject.Instantiate(PortPrefab);
//         Vector2OutPort portComponent = port.GetComponent<Vector2OutPort>();
//         portComponent.ParameterGetter = parameterGetter;
//         return portComponent;
//     }
//     public static EntityInPort EntityInputPort(GameEntity defaultValue)
//     {
//         GameObject port = GameObject.Instantiate(PortPrefab);
//         EntityInPort portComponent = port.GetComponent<EntityInPort>();
//         portComponent.defaultValue = defaultValue;
//         return portComponent;
//     }
//     public static EntityOutPort EntityOutputPort(Func<RefParameter<GameEntity>> parameterGetter)
//     {
//         GameObject port = GameObject.Instantiate(PortPrefab);
//         EntityOutPort portComponent = port.GetComponent<EntityOutPort>();
//         portComponent.ParameterGetter = parameterGetter;
//         return portComponent;
//     }

//     //return gameobject
//     public static GameObject FloatInputPort(float defaultValue, GameObject parent)
//     {
//         GameObject port = GameObject.Instantiate(PortPrefab, parent.transform);
//         floatInPort portComponent = port.GetComponent<floatInPort>();
//         portComponent.defaultValue = defaultValue;
//         return port;
//     }
//     public static GameObject FloatOutputPort(Func<Parameter<float>> parameterGetter, GameObject parent)
//     {
//         GameObject port = GameObject.Instantiate(PortPrefab, parent.transform);
//         floatOutPort portComponent = port.GetComponent<floatOutPort>();
//         portComponent.ParameterGetter = parameterGetter;
//         return port;
//     }
//     public static GameObject Vector2InputPort(Vector2 defaultValue, GameObject parent)
//     {
//         GameObject port = GameObject.Instantiate(PortPrefab, parent.transform);
//         Vector2InPort portComponent = port.GetComponent<Vector2InPort>();
//         portComponent.defaultValue = defaultValue;
//         return port;
//     }
//     public static GameObject Vector2OutputPort(Func<Parameter<Vector2>> parameterGetter, GameObject parent)
//     {
//         GameObject port = GameObject.Instantiate(PortPrefab, parent.transform);
//         Vector2OutPort portComponent = port.GetComponent<Vector2OutPort>();
//         portComponent.ParameterGetter = parameterGetter;
//         return port;
//     }
//     public static GameObject EntityInputPort(GameEntity defaultValue, GameObject parent)
//     {
//         GameObject port = GameObject.Instantiate(PortPrefab, parent.transform);
//         EntityInPort portComponent = port.GetComponent<EntityInPort>();
//         portComponent.defaultValue = defaultValue;
//         return port;
//     }
//     public static GameObject EntityOutputPort(Func<RefParameter<GameEntity>> parameterGetter, GameObject parent)
//     {
//         GameObject port = GameObject.Instantiate(PortPrefab, parent.transform);
//         EntityOutPort portComponent = port.GetComponent<EntityOutPort>();
//         portComponent.ParameterGetter = parameterGetter;
//         return port;
//     }

In [4]:
using System.Collections.Generic;
public class myClass{
    public int Ina;
}
List<myClass> a = new List<myClass>();
var t = new myClass();
t.Ina = 1;
a.Add(t);
if(a.Contains(t)){
System.Console.WriteLine( "tt");
}

tt
